# Visualizing Neptune Results

In [144]:
import neptune
import pandas as pd
import json 
from neptune.utils import align_channels_on_x
import plotly.graph_objects as go
from pandas.api.types import is_numeric_dtype, is_string_dtype
# Get project
project = neptune.init('cmbirmingham/turn-taking')

# Download experiments dashboard as pandas DataFrame
data = project.get_leaderboard(tag="pred: speaking")
data.head()

# Download project and select experiments
speaking_exps = project.get_experiments(tag='speaking')
my_exp = speaking_exps[1]

data_collected = my_exp.get_logs().keys()
keys_to_use = [k for k in data_collected if "f1" in k or "conf" in k]

print(my_exp.get_tags())
print(my_exp.get_parameters())
df = my_exp.get_numeric_channels_values(*keys_to_use)


,id,name,created,finished,owner,notes,running_time,size,tags,channel_best_so_far_run_score,...,parameter_classes,parameter_max_rolling,parameter_normalize,parameter_overlap,parameter_patience,parameter_pred_window,parameter_pruner,parameter_shuffle,parameter_trials,parameter_weight classes
0,TUR2-318,xgb_turn-taking,2021-01-23 22:38:52.825000+00:00,2021-01-24 00:52:23.494000+00:00,cmbirmingham,,8010,15686568,"[personal-laptop, pearson-m_hand-f, pred: spea...",0.8856320334990839,...,['speaking'],30.0,True,False,2.0,30.0,no pruning,False,15,True
1,TUR2-319,forest_turn-taking,2021-01-24 00:52:24.240000+00:00,2021-01-24 01:18:02.304000+00:00,cmbirmingham,,1538,15061023,"[personal-laptop, pearson-m_hand-f, pred: spea...",0.7959016393442623,...,['speaking'],30.0,True,False,2.0,30.0,no pruning,False,15,True
2,TUR2-320,tcn_turn-taking,2021-01-24 01:18:03.058000+00:00,2021-01-24 05:54:26.835000+00:00,cmbirmingham,,16583,56067288,"[personal-laptop, pearson-m_hand-f, pred: spea...",0.7702031602708803,...,['speaking'],30.0,True,False,2.0,30.0,no pruning,False,15,True
3,TUR2-321,tree_turn-taking,2021-01-24 05:54:27.691000+00:00,2021-01-24 06:12:05.003000+00:00,cmbirmingham,,1057,15012068,"[personal-laptop, pearson-m_hand-f, pred: spea...",0.7221812822402358,...,['speaking'],30.0,True,False,2.0,30.0,no pruning,False,15,True
4,TUR2-322,rnn_turn-taking,2021-01-24 06:12:05.879000+00:00,2021-01-24 08:25:35.035000+00:00,cmbirmingham,,8009,36514175,"[personal-laptop, pearson-m_hand-f, pred: spea...",0.652063599458728,...,['speaking'],30.0,True,False,2.0,30.0,no pruning,False,15,True


In [150]:

def get_text_channel(my_exp, channel_name):    
    channels_by_name = my_exp.get_channels()
    channel_id = channels_by_name[channel_name].id
    s = str(my_exp._backend.get_channel_points_csv(my_exp, channel_id).read())
    l = s.split("\n")[:-1]
    l = [i.split(",",1) for i in l]
    l = [[str(int(i[0][:-2])),i[1].replace("'",'"')] for i in l]
    l = {i[0]:json.loads(i[1]) for i in l}
    df = pd.DataFrame(l)
    df = df.T
    df.reset_index(inplace=True,drop=True)
    for c in df.columns:
        if is_numeric_dtype(df[c]):
            df[c] = pd.to_numeric(df[c])
    return df

def get_numeric_and_text(my_exp, keys_to_use=None):
    data_collected = my_exp.get_logs().keys()
    if not keys_to_use:
        keys_to_use = [k for k in data_collected if "f1" in k or "conf" in k]
    params_df = get_text_channel(my_exp, "run_parameters")
    results_df = my_exp.get_numeric_channels_values(*keys_to_use)
    final = pd.concat([results_df,params_df], axis=1)
    return final
# final = get_numeric_and_text(my_exp)


,x,val_speaking-conf-FN,val_speaking-conf-TP,val_speaking-f1-score,train_speaking-conf-FP,train_speaking-f1-score,test_speaking-conf-TN,val_speaking-conf-TN,train_speaking-conf-TN,test_speaking-conf-FN,...,train_speaking-conf-TP,train_speaking-conf-FN,max_depth,criterion,min_samples,window,max_features,r_win_size,step_size,sub_sample_neg_perc
0,0.0,3641.0,18873.0,0.756827,11256.0,0.896091,48598.0,47687.0,64284.0,3544.0,...,93642.0,10461.0,12,entropy,0.000119333,1,sqrt,28,5,75
1,1.0,250.0,3917.0,0.696603,8.0,0.972733,7060.0,7242.0,5819.0,394.0,...,17338.0,964.0,16,gini,0.000123911,27,auto,10,1,92
2,2.0,204.0,1458.0,0.746735,1417.0,0.821461,3387.0,3337.0,5108.0,332.0,...,5841.0,1122.0,11,gini,0.0342867,17,auto,16,4,74
3,3.0,540.0,5708.0,0.706479,9494.0,0.790942,11315.0,11406.0,21478.0,877.0,...,24747.0,3588.0,14,gini,0.0184381,9,auto,15,2,63
4,4.0,309.0,3710.0,0.730818,5999.0,0.810868,7425.0,7607.0,14998.0,519.0,...,16339.0,1623.0,8,entropy,0.00338956,14,auto,23,2,62
5,5.0,73.0,598.0,0.674944,1058.0,0.789518,1133.0,1166.0,1995.0,122.0,...,2320.0,179.0,4,entropy,0.00111929,28,log2,30,6,72
6,6.0,179.0,1773.0,0.717088,2763.0,0.803523,3556.0,3610.0,6812.0,279.0,...,7480.0,895.0,10,gini,0.00850271,29,log2,25,2,65
7,7.0,123.0,1002.0,0.765178,842.0,0.858503,2307.0,2316.0,3095.0,199.0,...,4059.0,496.0,5,gini,0.00331858,25,sqrt,15,4,78
8,8.0,38.0,864.0,0.653061,2652.0,0.715977,1176.0,1364.0,2883.0,63.0,...,3538.0,155.0,4,gini,0.10728,25,auto,6,5,57
9,9.0,81.0,738.0,0.720703,894.0,0.812618,1540.0,1540.0,2527.0,159.0,...,2808.0,401.0,11,gini,0.0162345,23,log2,9,6,74


In [166]:

def plot_par_cord(df, name='first_figure.html',color_metric='test_speaking-f1-score', columns_to_plot=None):
    dimensions = []
    if not columns_to_plot:
        columns_to_plot = df.columns
    for c in df.columns:
        if "conf" in c:
            continue
        elif is_numeric_dtype(df[c]):
            if "f1" in c or "auROC" in c or "AP" in c:
                r = [0,1]
            else:
                r = [df[c].min(),df[c].max()]
            dimensions.append(dict(
                label = c,
                range = r,
                values= df[c]
            ))
        elif is_string_dtype(df[c]):
            temp = pd.Categorical(df[c])
            dimensions.append(dict(
                label = c,
                tickvals = list(range(len(temp.categories.values))),
                ticktext = temp.categories.values,
                values = temp.codes
            ))


    
    fig = go.Figure(data=
        go.Parcoords(
            line = dict(color = df[color_metric],
                       colorscale = 'Electric',
                       showscale = True,
                       cmin = 0,
                       cmax = 1),
            dimensions = dimensions
        )
    )
    fig.write_html(name, auto_open=True)

In [168]:
# Tags of interest: 

speaking_exps = project.get_experiments(tag=['tcn','taking_yielding_holding_listening'])
for e in speaking_exps:
    print(e.get_tags())
my_exp = speaking_exps[-1]

data_collected = my_exp.get_logs().keys()
keys_to_use = [k for k in data_collected if "f1" in k or "conf" in k]

exp_df = get_numeric_and_text(my_exp, keys_to_use)

plot_par_cord(exp_df, color_metric='test_yielding-f1-score')

['Exp-1', 'pearson-m_hand-f', 'taking_yielding_holding_listening', '15 Trials', '23 Sessions', 'tcn']


In [132]:
len(t.categories.values[:])

2